In [ ]:
# question include new mode namley start_cv_run_building_id
# or introduces new parameter that can be use in the config file
# the latter is probably better
# pseudo code:
# start_cv_run_buidlingwise:
# if mode = group_cv_building
    # meter0 = train_df.meter == 0 returns logical vector
    # train_df = train_df meter0 rows
    # label = label aber nur meter 0 rows
# group kfold
# define the groups array by building_id
# and run the cross-validation

In [ ]:
# building_id keepen auch wenn dropped in make features
# if cv grouped on building id
# and in drop vector
# then delete it from drop vector

# summ we have a vector containing the building id
# this vector indicates which variables will be dropped
# but we nee building id to group the folds in the cross validation
# first solution see above
#
# also it needs to be dropped again in the cross validation function so taht it is
# not inculded in the parameeters for tehe mdoel

In [ ]:
# label = adden

In [2]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold

In [ ]:
def start_cv_run(train_df, label, params, splits, verbose_eval,
                 num_boost_round, early_stopping_rounds, output_filepath,
                 grouped_on_building):
    """
    Starts a Cross Validation Run with the parameters provided.
    Scores will be documented and models will be saved.
    :param train_df: DataFrame which contains the training data.
    :param label: A vector which contains the labels of the training data.
    :param params: Dictionary with the model parameters
    :param splits: Integer describing the number of folds / splitting fraction.
    :param verbose_eval: The interval where training information is printed
    to console.
    :param num_boost_round: Maximum number of rounds / estimators for the training.
    :param early_stopping_rounds: If no improvement of the validation score in
    n rounds occur, the training will be stopped.
    :param output_filepath: Directory that will contain the trained models.
    """
    if grouped_on_building:
        output_filepath = output_filepath + "grouped_cv"
        is_meter0 = (train_df.meter == 0).values
        train_df = train_df.iloc[ismeter0,]
        label = label.iloc[ismeter0,]
        groups = train_df.building_id
        gkf = GroupKFold(n_splits = splits)
        indices = gkf.split(train_df, label, groups)
    else:
        output_filepath = output_filepath + "_cv"
        kf = Kfold(n_splits = splits, shuffle = False, random = state = 1337)
        indices = kf.split(train_df, label)
    cv_results = []
    with timer("Performing " + str(splits) + " fold cross-validation"):
        kf = KFold(n_splits=splits, shuffle=False, random_state=1337)
        for i, (train_index, test_index) in enumerate(indices):
            with timer("~~~~ Fold %d of %d ~~~~" % (i + 1, splits)):
                x_train, x_valid = train_df.iloc[train_index], train_df.iloc[test_index]
                y_train, y_valid = label[train_index], label[test_index]

                train_lgb_df = lgb.Dataset(data=x_train, label=y_train)
                valid_lgb_df = lgb.Dataset(data=x_valid, label=y_valid)

                valid_sets = [train_lgb_df, valid_lgb_df]
                evals_result = dict()
                lgbm_model = lgb.train(params=params,
                                       train_set=train_lgb_df,
                                       num_boost_round=num_boost_round,
                                       valid_sets=valid_sets,
                                       valid_names=["train_loss", "eval"],
                                       verbose_eval=verbose_eval,
                                       evals_result=evals_result,
                                       early_stopping_rounds=early_stopping_rounds)
                save_model(output_filepath, lgbm_model)

                cv_results.append(evals_result)
        evaluate_cv_results(cv_results)


In [7]:
import numpy as np
from sklearn.model_selection import GroupKFold

In [8]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
groups = np.array([0, 0, 2, 2])
group_kfold = GroupKFold(n_splits=2)
group_kfold.get_n_splits(X, y, groups)

2

In [9]:
print(group_kfold)

GroupKFold(n_splits=2)


In [11]:
for train_index, test_index in group_kfold.split(X, y, groups):
    print("TRAIN:", train_index, "TEST", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [0 1] TEST [2 3]
TRAIN: [2 3] TEST [0 1]


In [17]:
 for a, (b, c) in enumerate(group_kfold.split(X, y, groups)):
        print([a, (b,c)])

[0, (array([0, 1]), array([2, 3]))]
[1, (array([2, 3]), array([0, 1]))]


In [18]:
group_kfold.split(X, y, groups)

<generator object _BaseKFold.split at 0x0000019A4925D548>

In [5]:
import yaml

In [6]:
with open("../src/config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile, Loader =yaml.FullLoader)

In [14]:
if 'building_id' in cfg["drop"]:
    
    cfg['drop'].pop('building_id')

TypeError: 'str' object cannot be interpreted as an integer

In [15]:
'building_id' in cfg["drop"]

SyntaxError: invalid syntax (<ipython-input-15-2d0d2196c6ba>, line 1)

In [16]:
if cfg['lgbm_cv_grouped_on_building']:
    print(2)

2


False